In [1]:
# IMPORTS
import os
import numpy as np
import time as time
import cv2

 

In [2]:
interval_of_interest = [0, 100]
second_interval_of_interest = [0, 100]
delta_of_interest = [-30, -10]
second_delta_of_interest = [10, 30]
RED = [0, 0, 255]
BLUE = [255, 0, 0]


In [8]:
def manipulate_frame(buf):
    # todo: try creating the mask from buf[0,:,:,1] so there is only one comparison and not 3, 
    # FIRST INTEREST (ATTACH)
    interval_mask_first_frame = np.where((buf[0,:,:, :1] >= interval_of_interest[0]) & (buf[0,:,:, :1]<interval_of_interest[1]))
    first_frame_vals = buf[0][interval_mask_first_frame[0], interval_mask_first_frame[1], interval_mask_first_frame[2]]
    next_frame_vals = buf[1][interval_mask_first_frame[0], interval_mask_first_frame[1], interval_mask_first_frame[2]]
    # FIRST INTEREST (DETACH)
    second_interval_mask_first_frame = np.where((buf[0] >= second_interval_of_interest[0]) & (buf[0]<second_interval_of_interest[1]))
    second_interest_first_frame_vals = buf[0][second_interval_mask_first_frame[0], second_interval_mask_first_frame[1], second_interval_mask_first_frame[2]]
    second_interest_next_frame_vals = buf[1][second_interval_mask_first_frame[0], second_interval_mask_first_frame[1], second_interval_mask_first_frame[2]]
    for k in range(len(interval_mask_first_frame[0])):
        delta_between_frames =  next_frame_vals[k] - first_frame_vals[k]  
        if delta_of_interest[0] <= delta_between_frames < delta_of_interest[1]:
            x, y, z = interval_mask_first_frame[0][k], interval_mask_first_frame[1][k], interval_mask_first_frame[2][k]
            buf[0][x][y] = np.array(RED, np.dtype('int64'))
    for j in range(len(second_interval_mask_first_frame[0])):
        delta_between_frames =  second_interest_next_frame_vals[j] - second_interest_first_frame_vals[j]  
        if second_delta_of_interest[0] <= delta_between_frames < second_delta_of_interest[1]:
            x, y, z = second_interval_mask_first_frame[0][j], second_interval_mask_first_frame[1][j], second_interval_mask_first_frame[2][j]
            buf[0][x][y] = np.array(BLUE, np.dtype('int64'))
    return buf   

def manipulate_single_video(file_name):    
    total_start = time.time()
    frames_in_memory = 2
    
    cap = cv2.VideoCapture("ForVideoAnalyze/{0}".format(file_name))
    
    final_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    out = cv2.VideoWriter("ForVideoAnalyze/results/{0}_{1}_{2}.avi".format(file_name[:-4], interval_of_interest, delta_of_interest),cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
    print("number of frames = {0}".format(final_frame_count))
    # new_video = np.zeros((final_frame_count, frame_height, frame_width, 3), np.dtype('int64'))
    buf = np.zeros((frames_in_memory, frame_height, frame_width, 3), np.dtype('int64'))
    ret = True
    
    fc = 0
    while fc < final_frame_count and ret:
        # loading the data according
        in_memory_frames_ctr = 0
        single_frame_start_time = time.time()
        while in_memory_frames_ctr < frames_in_memory:
            if np.sum(buf[frames_in_memory - 1]) > 0:
                temp = buf[frames_in_memory - 1].copy()
                # buf[FRAMES_IN_MEMORY-1] = np.zeros((frame_height, frame_width, 3))
                buf = np.zeros((frames_in_memory, frame_height, frame_width, 3), np.dtype('int64'))
                buf[0] = temp
            else:
                ret, frame = cap.read()
                buf[in_memory_frames_ctr] = frame
                fc += 1
            in_memory_frames_ctr += 1
        buf = manipulate_frame(buf)
        im = np.uint8(buf[0])
        out.write(im)
        # new_video[fc-2] = buf[0]
        single_frame_end_time = time.time()
        print("frame No:{0} has been manipulated.took:{1:.2} seconds".format(fc-2, single_frame_end_time - single_frame_start_time))
    
    out.release()
    cap.release()
    total_end = time.time()
    print("{0}: {1:.2f} minutes total".format(fc, (total_end - total_start) / 60))
    # os.system('afplay alarm.m4a')                


In [9]:
directory = os.fsencode("ForVideoAnalyze")

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith('.avi') or filename.endswith('.mp4'):
        print("file {0} has started".format(filename))
        manipulate_single_video(filename)

file exp89_PLT_coll4_CONTROL_SP1.avi has started
number of frames = 91


KeyboardInterrupt: 

In [11]:
a = np.array([[1,2],[1,2]])
b = np.array([[3,4],[3,4]])
c = np.concatenate((a,b))